In [7]:
import win32gui
import win32con
import time
import locator
import importlib
import ffbe_automator
importlib.reload(ffbe_automator)
importlib.reload(locator)

from ppadb.client import Client as AdbClient
#first one=62001
#second one=62025
my_client = AdbClient()
print(my_client.devices())
my_device = my_client.devices()[0]

# my_hwnd = win32gui.FindWindow(None, "facebook")
# my_hwnd = win32gui.FindWindow(None, "SM-N950N")
my_hwnd = win32gui.FindWindow(None, "SM-G950N")
print(my_hwnd)

[<ppadb.device.Device object at 0x00000289CD3FEBD0>, <ppadb.device.Device object at 0x00000289CD2FDF50>]
3081834


In [16]:
from enum import auto


importlib.reload(ffbe_automator)
automator = ffbe_automator.Automator()
automator.set_msg_handlers(print, print, print)
automator.list_all_methos()
automator.set_job('play_test')
automator.automation_by_job['play_test']()

Creating Automator.
['__init__', 'create_locator', 'elasped_time', 'init_automation_list', 'init_device', 'init_internal_vars', 'init_other', 'init_time', 'list_all_methos', 'play_multi', 'play_multi_client', 'play_quest', 'play_raid', 'play_raid_client', 'play_raid_host2', 'play_raid_host4', 'set_img_base_path', 'set_job', 'set_msg_handlers', 'set_user_params', 'set_window_and_device', 'skip_battle', 'start_automation', 'stop', 'stop_watch', 'summon', 'test']
Testing


In [8]:
device_name

'SM-M536S'

In [6]:
my_device.input_tap(300,300)

''

In [22]:
# debug msg
debug_msg_list = []
def debug(msg:str):
    if not msg in debug_msg_list:
        debug_msg_list.append(msg)
        print(msg)
def print_all_debug_msg():
    cnt = 0
    for m in debug_msg_list:
        print(f"{cnt}: {m}")
        cnt += 1

In [ ]:
# quest auto play
import datetime
# device_type = 'nox_1920_1080'
# device_type = 'nox_1280_720'
device_type = 'android'

if device_type == 'nox_1920_1080':
    my_locator = locator.Locator(my_hwnd, './1920_1080/quest/')
    my_locator.load_conf('nox_1920_1080')
elif device_type == 'nox_1280_720':
    my_locator = locator.Locator(my_hwnd, './1280_720/quest/')
    my_locator.load_conf('nox_1280_720')
elif device_type == 'android':
    my_locator = locator.Locator(my_hwnd, './1280_720/quest/')
    my_locator.load_conf('android')
else:
    pass
my_locator.confidence=0.85
my_locator.connect_click_method(my_device.input_tap)

for cnt in range(100):
    #퀘스트 자동 진행
    print(f"Starting quest automain. {cnt} times completed.")
    while not my_locator.locate('sortie'):
        my_locator.locate_and_click('select_chapter', xy='top_quest')
    ##skip 하기 누르기
    print("Before battle")
    while my_locator.locate('auto') == None:
        my_locator.locate_and_click('sortie')
        my_locator.click('story_skip1')
        time.sleep(2)
        my_locator.click('story_skip2')
    print("In battle stage")
    start_time = time.time()
    my_locator.locate('next', trial_number=600)
    print("Mission complete")
    time.sleep(10)
    elasped_time = time.time() - start_time
    print(f"elasped_time: {time.strftime('%M:%S', time.gmtime(elasped_time))}")
    print("Until end_of_quest")
    while my_locator.locate('end_of_quest') == None:
        my_locator.locate_and_click('next')
        my_locator.locate_and_click('close')
        my_locator.locate_and_click('later')
        my_locator.locate_and_click('no_evaluate')
    time.sleep(2)
    print("The quest ended")
    time.sleep(10)
    count = 0
    print(f"after batlle, until 'select chapter', repeating, ... story skip, count={count}")
    while my_locator.locate('select_chapter', trial_number=2) == None:
        my_locator.locate_and_click('end_of_quest')
        my_locator.locate_and_click('ok')
        my_locator.locate_and_click('later')
        my_locator.locate_and_click('close')
        if not my_locator.locate('select_chapter'):
            my_locator.click('story_skip1')
            time.sleep(1)
        if not my_locator.locate('select_chapter'):
            my_locator.click('story_skip3')
        count += 1
        my_locator.locate_and_click('story')

In [27]:
# multi auto play
import datetime
device_type = 'android'
rep_time = 20
num_of_players = 2
before_battle_cnt_limit = 40 * num_of_players

if device_type == 'nox_1920_1080':
    my_locator = locator.Locator(my_hwnd, './1920_1080/multi/')
    my_locator.load_conf('nox_1920_1080')
elif device_type == 'nox_1280_720':
    my_locator = locator.Locator(my_hwnd, './1280_720/multi/')
    my_locator.load_conf('nox_1280_720')
elif device_type == 'android':
    my_locator = locator.Locator(my_hwnd, './1280_720/multi/')
    my_locator.load_conf('android')
else:
    pass
my_locator.confidence=0.95
my_locator.connect_click_method(my_device.input_tap)

print("Starting multi automation")
for cnt in range(rep_time):
    print(f"Before battle stage. completed: {cnt} times")
    before_battle_cnt = 0
    print("Trial number of sortie: ", end='')
    sortie_trial_cnt = 1
    while not my_locator.locate('auto'):
        sortie_cond = False
        if num_of_players == 1:
            sortie_cond = True
        elif num_of_players == 2:
            if not my_locator.locate('one_person'):
                sortie_cond = True
        elif num_of_players == 3:
            if (my_locator.locate('three_people')) or (my_locator.locate('four_people')):
                sortie_cond = True
        elif num_of_players == 4:
            if (my_locator.locate('four_people')):
                sortie_cond = True
        if sortie_cond:
            print(f"{sortie_trial_cnt},", end='')
            my_locator.locate_and_click('sortie')
            sortie_trial_cnt += 1
        my_locator.locate_and_click('sortie_confirm')
        # my_locator.locate_and_click('not_auto')
        # my_locator.locate_and_click('ok')
        before_battle_cnt += 1
        if before_battle_cnt > 10:
             while my_locator.locate('checking_the_result'):
                my_locator.locate_and_click('checking_the_result')
                my_locator.locate_and_click('expel')
        if before_battle_cnt > before_battle_cnt_limit:
            print("Kicking all out!")            
            while(not my_locator.locate('one_person')):
                my_locator.locate_and_click('kick_out')
                my_locator.locate_and_click('expel')
            before_battle_cnt = 0
            sortie_trial_cnt
    print("In battle stage")
    start_time = time.time()
    # while not my_locator.locate('after_battle_next'):
    while not my_locator.locate('next'):
        my_locator.locate_and_click('ok')
        my_locator.locate_and_click('give_up')
        my_locator.locate_and_click('yes')
        time.sleep(1)
    # after battle stage
    print("after battle stage")
    elasped_time = time.time() - start_time
    print(f"elasped_time: {time.strftime('%M:%S', time.gmtime(elasped_time))}")
    while not my_locator.locate('organize'):
        my_locator.locate_and_click('next')
        # my_locator.locate_and_click('ok')
        my_locator.locate_and_click('cancel')
        my_locator.locate_and_click('go_back')

Starting multi automation
Before battle stage. completed: 0 times
Trial number of sortie: No such file: ./1280_720/multi/one_person.png
1,

KeyboardInterrupt: 

In [7]:
print(my_locator.locate('sortie'))
print(my_locator.locate_and_click('sortie'))

(1193, 655)
None
